In [ ]:
# !pip install selenium
# !apt-get update # Update package lists
# !apt-get install -y chromium-chromedriver # Install chromium and chromedriver

from selenium import webdriver
from selenium.webdriver.common.by import By # Import By
from datetime import datetime
import time
import csv
import pandas as pd

page = 1
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Run Chrome in headless mode
options.add_argument('--no-sandbox') # Disable sandbox for security reasons in this environment
# options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options) # Apply options to the driver

res_list = []

while True:
    driver.get(f'https://markets.businessinsider.com/news/nvda-stock?p={page}')
    ele_list = driver.find_elements(By.CLASS_NAME, 'latest-news__story')
    print('page(max 194): ',page)
    if page == 194:
        break

    for item in ele_list:
        date_str = item.find_element(By.CSS_SELECTOR, '.latest-news__meta time').get_attribute('datetime')
        
        event_str = str(item.find_element(By.CLASS_NAME, 'news-link').text)
        # print(event_str)

        timestamp = datetime.strptime(date_str, '%m/%d/%Y %H:%M:%S %p')
        timestamp = timestamp.strftime('%Y/%m/%d/')
        res_list.append([timestamp, event_str])

    page += 1

data = pd.DataFrame(res_list, columns=['timestamp', 'events'])

data.to_csv(r'data\raw\stocknews.csv' ,index=False ,encoding='utf-8', header=True)
    
driver.quit()

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
page = 1
options = webdriver.ChromeOptions()
options.add_argument('--headless') # Run Chrome in headless mode
options.add_argument('--no-sandbox') # Disable sandbox for security reasons in this environment
driver = webdriver.Chrome(options=options) # Apply options to the driver

res_list = []
count = 1
driver.get(r'https://finance.yahoo.com/quote/NVDA/history/?period1=917015400&period2=1728624531')
ele_list = driver.find_elements(By.TAG_NAME, 'tr')
stop = False
limit_date = datetime.strptime('2016/09/28', '%Y/%m/%d')
print('Furthest data obtained:',limit_date)
for row in ele_list:
    items = row.find_elements(By.TAG_NAME, 'td')

    item_list = []
    date = True
    item_count = 1
    for item in items:
        if date:
            text = item.text
            new_date = datetime.strptime(text, r'%b %d, %Y')
            print('data obtained:',new_date)
            if (new_date < limit_date):
                stop = True
                
            text = new_date.strftime('%Y/%m/%d')


            date = False

            item_list.append(text)
            item_count += 1
            continue
        if item_count == 7:
            volume_list = item.text.split(',')
            total = 0
            for i in range(len(volume_list)):
                total += (10**((len(volume_list) - i - 1)*3))*int(volume_list[i])
            item_list.append(total)
            continue

        item_count += 1
        item_list.append(item.text)
    if stop:
        break    
    res_list.append(item_list)
    count += 1

    
data = pd.DataFrame(res_list, columns=['date', 'open', 'high', 'low', 'close', 'Adj close', 'volume'])
data.to_csv(r'data\raw\stockPrice.csv', index=False, header=True, encoding='utf-8')
driver.quit()

In [3]:
import pandas as pd
df1 = pd.read_csv(r'D:\CODING\Project\NVIDIA Stock prediction\data\raw\stocknews.csv')
df2 = pd.read_csv(r'D:\CODING\Project\NVIDIA Stock prediction\data\raw\stockPrice.csv')

df1['timestamp'] = pd.to_datetime(df1['timestamp'])
df2['date'] = pd.to_datetime(df2['date'])
df_merged = pd.merge(df1, df2, left_on='timestamp', right_on='date', how='inner')
df_merged.to_csv(r'D:\CODING\Project\NVIDIA Stock prediction\data\raw\stock_news_price.csv', index=False, header=True, encoding='utf-8')

In [5]:
import pandas as pd
import ast

df = pd.read_csv(r'D:\CODING\Project\NVIDIA Stock prediction\data\processed\stock_news_price.csv')

df = df.groupby(['timestamp','open','high','low','close','Adj close','volume']).agg({
    'events': lambda x: list(x)
}).reset_index()
# df['events'] = df['events'].apply(ast.literal_eval)
df.to_csv(r'D:\CODING\Project\NVIDIA Stock prediction\data\processed\stock_news_price.csv', index=False)